In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C

# Tutorial for the YamboCalculator class

This tutorial describes the usage of the YamboCalculator class, for performing a single calculation
with the yambo package.

## Perform an Hartree-Fock computation for Silicon

__Follow the tutorial for the QeCalculator to produce the .save folder needed to generate the yambo SAVE.__

The first step needed to perform a Yambo computation is to generate the SAVE folder from a QuantumESPRESSO
computation. 

The mppi.Utilities module has a function that perform this task

In [3]:
from mppi import Utilities as U

In [4]:
source_dir = 'QeCalculator_test/si_nscf_test1.save'
run_dir = 'YamboCalculator_test'

In [5]:
U.build_SAVE(source_dir,run_dir)

Create folder YamboCalculator_test
Executing command: cd QeCalculator_test/si_nscf_test1.save; p2y -a 2
Executing command: ln -s /home/marco/Applications/MPPI/sphinx_source/tutorials/QeCalculator_test/si_nscf_test1.save/SAVE /home/marco/Applications/MPPI/sphinx_source/tutorials/YamboCalculator_test
Executing command: cd YamboCalculator_test;OMP_NUM_THREADS=1 yambo


Now the YamboInput class can create the yambo input object

In [6]:
inp = I.YamboInput(args='yambo -x -V rl',folder=run_dir)
inp

{'args': 'yambo -x -V rl',
 'folder': 'YamboCalculator_test',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [1591.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [9377.0, 'RL'],
  'VXCRLvcs': [9377.0, 'RL'],
  'QPkrange': [[1, 13, 1, 8], '']}}

We restrict the analysis to the first two kpoints

In [7]:
inp.set_kRange(1,2)
inp

{'args': 'yambo -x -V rl',
 'folder': 'YamboCalculator_test',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [1591.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [9377.0, 'RL'],
  'VXCRLvcs': [9377.0, 'RL'],
  'QPkrange': [[1, 2, 1, 8], '']}}

To run the computation we create an istance of YamboCalculator. This object behaves almost exactly as
the QeCalculator for what concerns the user interface

In [5]:
code = C.YamboCalculator(omp=1,mpi_run='mpirun -np 4',skip=True)

Initialize a Yambo calculator with OMP_NUM_THREADS=1 and command mpirun -np 4 yambo


In [9]:
code.global_options()

{'omp': 1,
 'mpi_run': 'mpirun -np 4',
 'executable': 'yambo',
 'skip': True,
 'verbose': True}

The structure of the folder in which yambo writes its results is governed by the name and jobname
variables. It is possible to provide only the name variable. 

The effect of this choice can be seen
in the command string executed by the calculator

In [10]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test1')
result

Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F hf_test1.in -J hf_test1 -C hf_test1


{'output': ['YamboCalculator_test/hf_test1/o-hf_test1.hf'],
 'ndb_folder': 'hf_test1'}

In this case yambo create the hf_test1 folder and write all the file in this folder. Result is a dictionary with the name of the
o- file and the folder that contains the ndb database.

Instead, if we set also set a jobname we have

In [11]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test2',jobname = 'hf_job_test2')
result

Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F hf_test2.in -J hf_job_test2 -C hf_test2


{'output': ['YamboCalculator_test/hf_test2/o-hf_job_test2.hf'],
 'ndb_folder': 'hf_job_test2'}

In this case yambo creates also the folder hf_job_test2 and write the database in this folder

Let's see what happens if skip is set to False but the directory with the same name of the one that yambo
is going to create is found

In [12]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test1',skip=False)
result

delete folder: YamboCalculator_test/hf_test1
Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F hf_test1.in -J hf_test1 -C hf_test1


{'output': ['YamboCalculator_test/hf_test1/o-hf_test1.hf'],
 'ndb_folder': 'hf_test1'}

The YamboCalculator delete the hf_test1 folder before running the computation. 

In the same way, if also a jobname is specified, the skip = False option provides

In [13]:
result = code.run(input = inp, run_dir = run_dir,name='hf_test2',jobname = 'hf_job_test2',skip=False)
result

delete folder: YamboCalculator_test/hf_test2
delete folder: YamboCalculator_test/hf_job_test2
Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F hf_test2.in -J hf_job_test2 -C hf_test2


{'output': ['YamboCalculator_test/hf_test2/o-hf_job_test2.hf'],
 'ndb_folder': 'hf_job_test2'}

## Perform a GW computation for Silicon

We make usage of the YamboCalculator to perform a different yambo computation. In this way we control
how this class manage the output files and the ndb database in various cases.

In [15]:
inp = I.YamboInput(args='yambo -d -k hartee -g n -p p -V qp',folder=run_dir)
inp.set_kRange(1,2)
#inp

In [16]:
result = code.run(input = inp, run_dir = run_dir,name='qp_test1',skip=False)
result

Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F qp_test1.in -J qp_test1 -C qp_test1


{'output': ['YamboCalculator_test/qp_test1/o-qp_test1.qp'],
 'ndb_folder': 'qp_test1'}

Perform the same computation but specify also a jobname 

In [17]:
result = code.run(input = inp, run_dir = run_dir,name='qp_test2',jobname = 'qp_job_test2')
result

Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F qp_test2.in -J qp_job_test2 -C qp_test2


{'output': ['YamboCalculator_test/qp_test2/o-qp_job_test2.qp'],
 'ndb_folder': 'qp_job_test2'}

## Test of the ExtendOut option in the input file

The ExtendOut option enables the writing of all the variables in the output files of Yambo. 
This feature has no effect for an HF computation and we test for a QP one:

In [16]:
inp = I.YamboInput(args='yambo -d -k hartee -g n -p p -V qp',folder=run_dir)
inp.set_kRange(1,2)
inp.set_extendOut()
#inp

In [15]:
result = code.run(input = inp, run_dir = run_dir,name='qp_test_ExtendOut')
result

Run directory YamboCalculator_test
Executing command: mpirun -np 4 yambo -F qp_test_ExtendOut.in -J qp_test_ExtendOut -C qp_test_ExtendOut


{'output': ['YamboCalculator_test/qp_test_ExtendOut/o-qp_test_ExtendOut.qp'],
 'ndb_folder': 'qp_test_ExtendOut'}

## Perform a ypp computation

__Follow the tutorial for the QeCalculator to produce the .save folder needed to generate the yambo SAVE.__

The YamboCalculator class can manage also ypp computation.

Let's see an example by performing a band calculation along a path

In [5]:
inp = I.YamboInput(args='ypp -s b',folder=run_dir,filename='ypp.in') 
inp

{'args': 'ypp -s b',
 'folder': 'YamboCalculator_test',
 'filename': 'ypp.in',
 'arguments': [],
 'variables': {'OutputAlat': [0.0, ''],
  'INTERP_Shell_Fac': [20.0, ''],
  'BANDS_steps': [10.0, ''],
  'INTERP_mode': 'NN',
  'cooIn': 'rlu',
  'cooOut': 'rlu',
  'CIRCUIT_E_DB_path': 'none',
  'BANDS_bands': [[1, 8], ''],
  'INTERP_Grid': [['-1', '-1', '-1'], '']}}

We define a calculator for ypp. This calculation requires 1 mpirun (see yambo for further information)

In [6]:
code = C.YamboCalculator(mpi_run='mpirun -np 1',executable='ypp',skip=False)
code.global_options()

Initialize a Yambo calculator with OMP_NUM_THREADS=1 and command mpirun -np 1 ypp


{'omp': 1,
 'mpi_run': 'mpirun -np 1',
 'executable': 'ypp',
 'skip': False,
 'verbose': True}

Set the input parameter to perform the band computation along a path

In [7]:
# in alat
G = [0.,0.,0.]
X = [1.,0.,0.]
L = [0.5,0.5,0.5]
K = [1.0,0.5,0.]
path = [L,G,X,K,G]

band_range = [2,5]
bands_step = 30

In [8]:
# scissor
# inp['variables']['GfnQP_E'] = [1.0,1.0,1.0]

# band structure
# Some methods that perform these operation can be added in the YamboInput class
inp['variables']['BANDS_steps'] = [bands_step,'']
inp['variables']['BANDS_bands'] = [band_range,'']
inp['variables']['BANDS_kpts'] = [path,'']
inp['variables']['cooIn'] = 'alat'
inp['variables']['cooOut'] = 'alat'
inp

{'args': 'ypp -s b',
 'folder': 'YamboCalculator_test',
 'filename': 'ypp.in',
 'arguments': [],
 'variables': {'OutputAlat': [0.0, ''],
  'INTERP_Shell_Fac': [20.0, ''],
  'BANDS_steps': [30, ''],
  'INTERP_mode': 'NN',
  'cooIn': 'alat',
  'cooOut': 'alat',
  'CIRCUIT_E_DB_path': 'none',
  'BANDS_bands': [[2, 5], ''],
  'INTERP_Grid': [['-1', '-1', '-1'], ''],
  'BANDS_kpts': [[[0.5, 0.5, 0.5],
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [1.0, 0.5, 0.0],
    [0.0, 0.0, 0.0]],
   '']}}

In [12]:
result = code.run(run_dir=run_dir,input=inp,name='bands_test1')
result

delete folder: YamboCalculator_test/bands_test1
Run directory YamboCalculator_test
Executing command: mpirun -np 1 ypp -F bands_test1.in -J bands_test1 -C bands_test1


{'output': ['YamboCalculator_test/bands_test1/o-bands_test1.bands_interpolated'],
 'ndb_folder': 'bands_test1'}

Result can be parsed using the YamboParser class of this package or other external tools

Test the jobname specification for this type of computation

In [13]:
result = code.run(run_dir=run_dir,input=inp,name='bands_test2',jobname='bands_job_test2')
result

delete folder: YamboCalculator_test/bands_test2
delete folder: YamboCalculator_test/bands_job_test2
Run directory YamboCalculator_test
Executing command: mpirun -np 1 ypp -F bands_test2.in -J bands_job_test2 -C bands_test2


{'output': ['YamboCalculator_test/bands_test2/o-bands_job_test2.bands_interpolated'],
 'ndb_folder': 'bands_job_test2'}